In [11]:
import os

labels_dir = "D:/intezet/Bogi/Yolo/combined_dataset/labels"
for subfolder in os.listdir(labels_dir):
    subfolder_path = os.path.join(labels_dir, subfolder)
    if os.path.isdir(subfolder_path):
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".png") or filename.endswith(".txt"):
                # Remove '_combined' from the filename (before extension)
                new_name = filename.replace("_combined", "")
                # If you want to keep the .png extension, replace .txt with .png
                new_name = os.path.splitext(new_name)[0] + ".txt"
                src = os.path.join(subfolder_path, filename)
                dst = os.path.join(subfolder_path, new_name)
                os.rename(src, dst)

In [2]:
from ultralytics import YOLO
import torch.nn as nn

# Load the pre-trained model
model = YOLO('yolov8s.pt')

# Get the first layer (which is a Conv module in YOLOv8)
first_layer = model.model.model[0]

# Check if it's a Conv module and modify its Conv2d layer
if hasattr(first_layer, 'conv'):
    conv_layer = first_layer.conv
    if isinstance(conv_layer, nn.Conv2d):
        # Modify input channels to 6
        conv_layer.in_channels = 6
        # Adjust the weight shape
        if conv_layer.weight.shape[1] != 6:
            import torch
            new_weight = torch.zeros(conv_layer.out_channels, 6, *conv_layer.weight.shape[2:])
            torch.nn.init.xavier_uniform_(new_weight)
            conv_layer.weight = nn.Parameter(new_weight)
        print("Successfully modified input channels to 6")
    else:
        raise ValueError("Conv layer inside Conv module is not a Conv2d layer")
else:
    raise ValueError("First layer does not have a conv attribute")

# Save the modified model
model.save('yolov8s_6ch.pt')
print("Model saved as yolov8s_6ch.pt")

Successfully modified input channels to 6
Model saved as yolov8s_6ch.pt


In [23]:
from custom_dataset import TwoChannelDataset
print("Custom dataset class imported successfully:", TwoChannelDataset)

Custom dataset class imported successfully: <class 'custom_dataset.TwoChannelDataset'>


In [ ]:
import os
from ultralytics import YOLO
from custom_trainer import CustomTrainer

print("Starting training script...")
print(f"Current working directory: {os.getcwd()}")

model = YOLO('yolov8s_6ch.pt')
print("Model loaded successfully")

# Clear cache
os.remove('D:/intezet/Bogi/Yolo/combined_dataset/labels/train/original.cache')
os.remove('D:/intezet/Bogi/Yolo/combined_dataset/labels/val/original.cache')

model.train(
    data='D:/intezet/Bogi/Yolo/combined_dataset/data.yaml',
    epochs=40,
    imgsz=640,
    batch=16,
    patience=5,
    device='cpu',
    trainer=CustomTrainer
)

Starting training script...
Current working directory: d:\intezet\Bogi\Yolo\combined_dataset
Custom dataset class: <class 'custom_dataset.TwoChannelDataset'>
Custom trainer class: <class 'custom_trainer.CustomTrainer'>
Model loaded successfully
Ultralytics 8.3.152  Python-3.12.4 torch-2.7.1+cpu CPU (Intel Core(TM) 5 120U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/intezet/Bogi/Yolo/combined_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, 

TypeError: BaseDataset.__init__() got an unexpected keyword argument 'data'